In [98]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.model_selection import train_test_split
from ucimlrepo import fetch_ucirepo

pd.set_option('display.width', 2000)

In [90]:
# fetch dataset
heart_disease = fetch_ucirepo(id=45)

X = heart_disease.data.features
Y = heart_disease.data.targets

# Convert target to binary
Y = Y['num'].apply(lambda x: 1 if x > 0 else 0)

# Join features and target
X['target'] = Y
# Remove records with missing values
X = X[~np.isnan(X).any(axis=1)]

In [3]:
def one_hot_encode(df, column, column_names):
    dummies = pd.get_dummies(df[column], prefix=column)
    column_names = [column + '_' + str(name) for name in column_names]
    dummies.columns = column_names
    dummies = dummies.astype('int64')
    df = pd.concat([df, dummies], axis=1)
    df.drop(column, axis=1, inplace=True)
    return df

In [4]:
X =  one_hot_encode(X, 'cp', ['typical_angina', 'atypical_angina', 'non-anginal_pain', 'asymptomatic'])
X = one_hot_encode(X, 'thal', ['normal', 'ST-T_wave_abnormality', 'left_ventricular_hypertrophy'])
X = one_hot_encode(X, 'slope', ['upsloping', 'flat', 'downsloping'])
X = one_hot_encode(X, 'restecg', ['normal', 'fixed_defect', 'reversable_defect'])

In [91]:
# Extract target without missing values
Y = X['target']
X = X.drop('target', axis=1)

# Normalize data to [0, 1] range
X_norm = (X - X.min()) / (X.max() - X.min())

In [96]:
class Neuron():
    def __init__(self, num_of_weights, stand_dev=1):
        self.weights = np.random.normal(scale=stand_dev, size=num_of_weights)
        self.bias = np.random.normal(scale=stand_dev)
        self.X = None
        self.derivative = None

    @staticmethod
    def sigmoid(X):
        return np.exp(-np.logaddexp(0, -X))

    def sigmoid_derivative(self, X):
        return self.sigmoid(X) * (1 - self.sigmoid(X))

    def forward(self, inputs):
        self.X = inputs
        return self.sigmoid(np.dot(inputs, self.weights) + self.bias)

    def backward(self, error, weights_next_layer=None):
        if weights_next_layer is not None:
            error = error.T @ weights_next_layer
        self.derivative = error * self.sigmoid_derivative(np.dot(self.X,self.weights) + self.bias)
        return self.derivative
    
    def update(self, learning_rate):
        self.weights -=  learning_rate * np.dot(self.X.T, self.derivative)
        self.bias -= learning_rate * np.sum(self.derivative)
        self.X = None
        self.derivative = None

class NeuralNetwork():
    def __init__(self, num_of_inputs, hidden_layers, num_of_outputs, stand_dev=1.0):
        self.layers = []
        self.num_of_inputs = num_of_inputs
        self.hidden_layers = hidden_layers
        self.num_of_outputs = num_of_outputs
        self.num_of_hidden_layers = len(hidden_layers)
        self.stand_dev = stand_dev
        
        self.layers.append([Neuron(num_of_inputs, self.stand_dev) for _ in range(hidden_layers[0])])
        for i in range(1, self.num_of_hidden_layers):
            self.layers.append([Neuron(hidden_layers[i-1], self.stand_dev) for _ in range(hidden_layers[i])])
            if self.num_of_outputs == 1:
                self.layers.append([Neuron(hidden_layers[-1], self.stand_dev)])

    def cross_entropy(self, y, y_pred):
        return -np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred), axis=1)

    def cross_entropy_derivative(self, y, y_pred):
        return -np.sum(y / y_pred - (1 - y) / (1 - y_pred), axis=1)

    def forward(self, X):
        for layer in self.layers:
            X = np.array([neuron.forward(X) for neuron in layer]).T
        return X        

    def backward(self, output_error):
        # Iterate over the hidden layers in reverse order and calculate their gradients
        output_layer = self.layers[-1]
        output_error = np.array([neuron.backward(output_error) for neuron in output_layer])
        weights_next_layer = np.array([neuron.weights for neuron in output_layer]).T

        for layer in reversed(self.layers[:-1]):
            output_error = np.array([neuron.backward(output_error, weights_next_layer[index]) for index, neuron in enumerate(layer)])
            weights_next_layer = np.array([neuron.weights for neuron in layer]).T
        
        # Return the final gradients
        return output_error

    def update(self, learning_rate):
        for layer in self.layers:
            for neuron in layer:
                neuron.update(learning_rate)

    def fit(self, X, y, epochs, batch_size, learning_rate):
        n_samples, n_features = X.shape
        # Jeśli batch_size jest większy niż liczba próbek, to ustawiamy go na liczbę próbek -> zwyczajnie uczymy na całym zbiorze GD
        if batch_size > n_samples:
            batch_size = n_samples
        cost_list = []
        for epoch in range(epochs):
            random_order = np.random.permutation(n_samples)
            X_shuffled = X.values[random_order]
            y_shuffled = y.values[random_order]
            # lista kosztów w paczkach dla każdej epoki
            cost_list_in_batch = []

            for batch_index in range(0, n_samples, batch_size):

                X_batch = X_shuffled[batch_index:batch_index + batch_size]
                y_batch = y_shuffled[batch_index:batch_index + batch_size].reshape(-1, 1)
                predictions = self.forward(X_batch)
                output_error = self.cross_entropy_derivative(y_batch, predictions)
                self.backward(output_error)
                self.update(learning_rate)
                
            cost = self.cross_entropy(y_batch, predictions)
            cost_list.append(np.mean(cost))

            if epoch % (epochs // 4) == 0:
                print(f'Epoch: {epoch}, Loss: {np.mean(cost)}')

            #jeśli koszt jest mniejszy niż epsilon to stop
            # if(i > 0 and abs(cost - self.cost_list[-1]) < self.epsilon ):
                # break
            # self.cost_list.append(cost)
            # iteration_cost = np.mean(cost_list_in_batch)
            # self.mean_cost_list.append(iteration_cost)
            # self.epoch_list.append(i)
        print(f'Epoch: {epoch}, Loss: {np.mean(cost)}')


    def predict(self, X):
        return self.forward(X)

In [97]:
seed = 42
np.random.seed(seed)
X_train, X_test, Y_train, Y_test = train_test_split(X_norm, Y, test_size=0.2)
print(X_train.shape, Y_train.shape)
mlp = NeuralNetwork(num_of_inputs=X_train.shape[1], hidden_layers=[10, 10], num_of_outputs=1)
mlp.fit(X_train, Y_train, epochs=1000, batch_size=40, learning_rate=0.002)

(237, 13) (237,)
Epoch: 0, Loss: 0.8260260462120032
Epoch: 250, Loss: 0.3696522400110217
Epoch: 500, Loss: 0.2836277138829489
Epoch: 750, Loss: 0.3555214253241455
Epoch: 999, Loss: 0.4243593937986521
